In [98]:
import numpy
import random

<h2>Some Good Function</h2>

In [99]:
def gray_to_integer(array_number):
    binary_string = ''.join(array_number)
    gray_code = int(binary_string, 2)
    number = gray_code
    for i in range(1, len(array_number)):
        number ^= (gray_code >> i)
    return number

def integer_to_gray(number):
    n=int(number)
    gray_code = n ^ (n >> 1)
    binary_string = bin(gray_code)[2:].zfill(4)
    binary_code = [str(bit) for bit in binary_string]
    return binary_code

def Allnum_to_gray(number):
    covert=[]
    if number>=0:
        covert.append('1')
    else:
        covert.append('0')
    num=str(number)    
    correct_part,ashar_part=num.split('.')
    covert.extend(integer_to_gray(correct_part))  
    covert.extend(integer_to_gray(ashar_part))
    return covert  


def get_number(chrom):
  sign = chrom[0]
  correct_part = chrom[1:5]
  ashari_part = chrom[5:9]
  num = ''
  if sign == '0':
    num += '-'
  
  num +=(str(gray_to_integer(correct_part))+'.'+ \
            str(gray_to_integer(ashari_part)))
  n=float(num)
  return n




<h2>Define Genetic Algorithm</h2>

In [100]:
class Genetic_Algorithm:
  def __init__(self, size_population,rate_elitist,generations,rate_mutation,
              function,chro_len):
    self.rate_mutation = rate_mutation
    self.population = []
    self.size_population = size_population
    self.rate_elitist = rate_elitist
    self.generations = generations
        
    self.min_root=-9
    self.max_root=9
    self.function=function
    self.chro_len=chro_len
  
  def generate_chromosome(self):
    n = random.uniform(self.min_root, self.max_root)
    n =Allnum_to_gray(round(n, 1))
    return n
  
  def generate_population(self):
    for i in range(self.size_population):
      chro = self.generate_chromosome()
      self.population.append(chro)
      
  def fitness(self, chro):
    n = get_number(chro)
    y = self.function(n)
    return 1 / (1 + abs(y))

  def sort_population(self):
    return sorted(self.population, reverse=True, key=self.fitness)
  
  def select_population(self, ranked_pop):
    pool = []
    pop_size = len(ranked_pop)
    elitist_size = int(pop_size * self.rate_elitist)
    remain_size = pop_size - elitist_size

    # Select elitist individuals
    pool.extend(ranked_pop[:elitist_size])

    # Calculate cumulative probabilities
    fits = [self.fitness(e) for e in ranked_pop]
    cum_probs = [sum(fits[:i+1]) for i in range(pop_size)]
    sum_fits = cum_probs[-1]

    # Select remaining individuals using roulette wheel selection
    for _ in range(remain_size):
        pick = random.uniform(0, sum_fits)
        for i in range(pop_size):
            if pick <= cum_probs[i]:
                pool.append(ranked_pop[i])
                break

    return pool
  

  def crossover(self, parent1, parent2):
    size = len(parent1)
    cross_point = int(random.random() * size)
    child_part1 = parent1[:cross_point]
    child_part2 = parent2[cross_point:]
    child = child_part1 + child_part2
    return child

  def mutatation(self, chro):
    size = len(chro)
    new = chro
    for i in range(size):
      if random.random() < self.rate_mutation:
        new[i] = str(1 - int(chro[i]))
    return new

  def crossover_population(self, pool):
    new_pop = []
    pop_size = len(pool)
    elitist_size = int(pop_size * self.rate_elitist)
    remain_size = pop_size - elitist_size
    new_pop.extend(pool[:elitist_size])
    for _ in range(remain_size):
      parent1, parent2 = random.choices(pool, k=2)
      child = self.crossover(parent1, parent2)
      new_pop.append(child)

    return new_pop

  def mutate_population(self, pool):
    new_pop=[]
    for i in range(len(pool)):
      mutated = self.mutatation(pool[i])
      new_pop.append(mutated)
    return new_pop

  def new_population(self, new_pop):
    self.population = new_pop

  def next_generation(self):
    ranked_pop = self.sort_population()
    pool = self.select_population(ranked_pop)
    crossovered_pool = self.crossover_population(pool)
    mutated_pool = self.mutate_population(crossovered_pool)
    self.new_population(mutated_pool)    
    # returned before new generation
    best_chromosome = ranked_pop[0]
    avg_fitness = sum([self.fitness(e) for e in ranked_pop]) / len(ranked_pop)
    return best_chromosome, avg_fitness

  def run(self):
    self.generate_population()
    best_chromosomes = []
    avg_fitness = []
    for i in range(self.generations):
      best, avg = self.next_generation()
      best_chromosomes.append(best)
      avg_fitness.append(avg)
      self.print_log(i, avg, best, get_number(best), self.fitness(best))
    
    ranked_pop = self.sort_population()
    best = ranked_pop[0]
    avg = sum([self.fitness(e) for e in ranked_pop]) / len(ranked_pop)
    best_chromosomes.append(best)
    avg_fitness.append(avg)
    self.print_log(self.generations, avg, best, get_number(best), self.fitness(best))

    return best_chromosomes, avg_fitness

  def print_log(self, i, avg, best, bestroot, bestfit):
    print('Generation:', i)
    print('Best Chromosomes:', best)
    print('Best Root:', bestroot)
    print('Fitness:', bestfit)
    print('********************************')

In [101]:
size_population = 50
rate_elistic = 0.1
generation = 50
rate_mutation = 0.005

In [102]:
FUNC = lambda x: 2 *(x) - 4
model = Genetic_Algorithm(size_population,rate_elistic,generation, rate_mutation,FUNC,9)
bests, avg_fitness = model.run()
bests_fitness = numpy.array([model.fitness(e) for e in bests])

Generation: 0
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '0', '0', '1']
Best Root: 2.1
Fitness: 0.8333333333333333
********************************
Generation: 1
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '0', '0', '0']
Best Root: 2.0
Fitness: 1.0
********************************
Generation: 2
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '0', '0', '0']
Best Root: 2.0
Fitness: 1.0
********************************
Generation: 3
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '0', '0', '0']
Best Root: 2.0
Fitness: 1.0
********************************
Generation: 4
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '1', '0', '0']
Best Root: 2.7
Fitness: 0.41666666666666663
********************************
Generation: 5
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '0', '0', '0']
Best Root: 2.0
Fitness: 1.0
********************************
Generation: 6
Best Chromosomes: ['1', '0', '0', '1', '1', '0', '0', '0', '0']
Best Root: 2.0
Fitness: 1.0
*****************************

In [103]:
FUNC = lambda x: (x)**2 -8*(x) +4
model = Genetic_Algorithm(size_population,rate_elistic,generation, rate_mutation,FUNC,9)
bests, avg_fitness = model.run()
bests_fitness = numpy.array([model.fitness(e) for e in bests])

Generation: 0
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '1', '1']
Best Root: 0.5
Fitness: 0.8
********************************
Generation: 1
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '1', '1']
Best Root: 0.5
Fitness: 0.8
********************************
Generation: 2
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '1', '1']
Best Root: 0.5
Fitness: 0.8
********************************
Generation: 3
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '1', '1']
Best Root: 0.5
Fitness: 0.8
********************************
Generation: 4
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '0', '1']
Best Root: 0.6
Fitness: 0.6944444444444446
********************************
Generation: 5
Best Chromosomes: ['1', '0', '1', '0', '0', '0', '1', '1', '1']
Best Root: 7.5
Fitness: 0.8
********************************
Generation: 6
Best Chromosomes: ['1', '0', '1', '0', '0', '0', '1', '1', '1']
Best Root: 7.5
Fitness: 0.8
********************************
Generation: 

In [104]:
FUNC = lambda x: 4*(x)**3 -5*(x)**2+(x) -1
model = Genetic_Algorithm(size_population,rate_elistic,generation, rate_mutation,FUNC,9)
bests, avg_fitness = model.run()
bests_fitness = numpy.array([model.fitness(e) for e in bests])

Generation: 0
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '0', '1']
Best Root: 1.1
Fitness: 0.615006150061501
********************************
Generation: 1
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '1', '1']
Best Root: 1.2
Fitness: 0.9191176470588233
********************************
Generation: 2
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '1', '1']
Best Root: 1.2
Fitness: 0.9191176470588233
********************************
Generation: 3
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '1', '1']
Best Root: 1.2
Fitness: 0.9191176470588233
********************************
Generation: 4
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '1', '1']
Best Root: 1.2
Fitness: 0.9191176470588233
********************************
Generation: 5
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '1', '1']
Best Root: 1.2
Fitness: 0.9191176470588233
********************************
Generation: 6
Best Chromosomes: ['1', '0', '0', '0', '1', '0', '0', '1', '1']

In [105]:
FUNC = lambda x: 186*(x)**3 -7.22*(x)**2+15.5*(x) -13.2
model = Genetic_Algorithm(size_population,rate_elistic,generation, rate_mutation,FUNC,9)
bests, avg_fitness = model.run()
bests_fitness = numpy.array([model.fitness(e) for e in bests])

Generation: 0
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '0', '1', '0']
Best Root: 0.3
Fitness: 0.19313221831665953
********************************
Generation: 1
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '0', '1', '0']
Best Root: 0.3
Fitness: 0.19313221831665953
********************************
Generation: 2
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '0', '1', '0']
Best Root: 0.3
Fitness: 0.19313221831665953
********************************
Generation: 3
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '0', '1', '0']
Best Root: 0.3
Fitness: 0.19313221831665953
********************************
Generation: 4
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '0', '1', '0']
Best Root: 0.3
Fitness: 0.19313221831665953
********************************
Generation: 5
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '1', '0']
Best Root: 0.4
Fitness: 0.21057951482479773
********************************
Generation: 6
Best Chromosomes: ['1', '0', '0', '0', '0', '0', '1', '1